# Deep Learning - Winter Term 2023/2024

<hr style="border:2px solid gray">

### Exercise Sheet 01

<br>

## 3.1) Autograd Implementation

In the first exercise sheet, we will puzzle out the backpropagation in autograd. We believe, it will be fruitful for you, if you solve the first exercises and obtain an intution about how a computational graph works before you start the programming exercise. Here, you basically need to implement derivatives of certain function. Be careful with reoccurence of the same variable though!

- We didn't write any tests for you because you can do it yourself! For instance, you could write down any equation you want and compute the gradients per hand and test your code.
- ***Note:*** Try to think about the edge cases in your implementation.

In [ ]:
# We will only use NumPy for this exercise
import numpy as np

In [ ]:
'''
As you probably know, Python is an object-oriented programming language.
Therefore we will treat each node in the computational graph as an object.
Here is the Node class, that creates object with certain information (i.e. scalar, name etc.)
Each Node is an object and represents a node in the computational graph,
whereas the comp. graph itself is a graph structure, composed of multiple nodes arranged in a systematic fashion.
'''

class Node:
    
    ### Initialization of the class ###
    '''
    scalar: given scalar -- only scalars will be used in this exercise
    _children: child node in the graph ##_ in the beginning means it is an internal variable for the graph construction##
    _op: applied operation in order to reach the scalar
    name: string representation of the scalar
    '''
    def __init__(self, scalar, _children=(), _op='', name=''):
        self.scalar = scalar
        # Before computing the gradients, you always need to make sure that
        # they are set to zero. To do that, you can basically initialize the nodes again.
        self.grad = 0.0 # By default we assume the variable does not effect the function
        self.name = name
        self._backward = lambda: None # Meaning the function doesn't do anything by default
        self._prev = set(_children) # Save the child node in a set.
        self._op = _op
    
    # This function just makes the printing of the Node human-readable
    def __repr__(self):
        return f"Node(name={self.name}, scalar={self.scalar}, grad={self.grad})"
    
    def __add__(self, other):
        other = other if isinstance(other, Node) else Node(other)
        out = Node(self.scalar + other.scalar, (self, other), '+') # forward pass
        
        def _backward():
            # TODO: implement the backward pass of this function
            raise NotImplementedError()
        out._backward = _backward
        
        return out
    
    def __mul__(self, other):
        other = other if isinstance(other, Node) else Node(other)
        out = Node(self.scalar * other.scalar, (self, other), '*')
        
        def _backward():
            # TODO: implement the backward pass of this function
            raise NotImplementedError()
        out._backward = _backward
        
        return out
        
    def __pow__(self, other):
        other = other if isinstance(other, Node) else Node(other)
        out = Node(self.scalar**other.scalar, (self, other), f'**{other.scalar}')

        def _backward():
            # TODO: implement the backward pass of this function (be careful with the edge cases!)
            raise NotImplementedError()
        out._backward = _backward
        
        return out
    
    def exp(self):
        x = self.scalar
        out = Node(np.exp(x), (self, ), 'exp')
    
        def _backward():
            # TODO: implement the backward pass of this function
            raise NotImplementedError()
        out._backward = _backward
    
        return out

    def sigmoid(self):
        # TODO: implement the forward pass of this function
        raise NotImplementedError()
    
        def _backward():
            # TODO: implement the backward pass of this function
            raise NotImplementedError()
        out._backward = _backward
    
        return out
    
    def tanh(self):
        # TODO: implement the forward pass of this function
        raise NotImplementedError()
    
        def _backward():
            # TODO: implement the backward pass of this function
            raise NotImplementedError()
        out._backward = _backward
    
        return out
    
    def relu(self):
        # TODO: implement the forward pass of this function
        raise NotImplementedError()

        def _backward():
            # TODO: implement the backward pass of this function
            raise NotImplementedError()
        out._backward = _backward

        return out
    
    def log(self):
        x = self.scalar
        out = Node(np.nan if x < 0 else np.log(x), (self, ), 'log')
        
        def _backward():
            # TODO: implement the backward pass of this function
            raise NotImplementedError()
        out._backward = _backward
        
        return out
    
    def sqrt(self):
        assert self.scalar > 0, "No complex spaces here"
        x = self.scalar
        out = Node(np.sqrt(x), (self, ), 'sqrt')
        
        def _backward():
            # TODO: implement the backward pass of this function
            raise NotImplementedError()
        out._backward = _backward
        
        return out
    
    def cos(self):
        x = self.scalar
        out = Node(np.cos(x), (self, ), 'cos')
        
        def _backward():
            # TODO: implement the backward pass of this function
            raise NotImplementedError()
        out._backward = _backward
        
        return out
    
    def sin(self):
        x = self.scalar
        out = Node(np.sin(x), (self, ), 'sin')
        
        def _backward():
            # TODO: implement the backward pass of this function
            raise NotImplementedError()
        out._backward = _backward
        
        return out
    
    # This function performs backpropagation for each node
    # Nothing to implement here but you can spend some time to understand how
    # the algorithm sorts the nodes.
    def backward(self):
        # topological order of the visited children in the graph
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        # go one variable at a time and apply the chain rule to get its gradient
        # The gradient of the last operation wrt. itself is always 1.
        # We set this first gradient here.
        self.grad = 1.0
        for v in reversed(topo):
            v._backward()
    
    # Below are same necessary trivial definitions.
    # Feel free to use these functions. No gradient computation
    # is needed as they are implicitely implemented in prev. functions.
    def __neg__(self): # -self
        return self * -1

    def __radd__(self, other): # other + self
        return self + other

    def __sub__(self, other): # self - other
        return self + (-other)

    def __rsub__(self, other): # other - self
        return other + (-self)

    def __rmul__(self, other): # other * self
        return self * other

    def __truediv__(self, other): # self / other
        return self * other**-1

    def __rtruediv__(self, other): # other / self
        return other * self**-1
    
# ----------------------------------------------------------------
# Helper functions
# ----------------------------------------------------------------
    
def trace_nodes(node):
    '''
    Builds a list of all nodes of a graph. The method is 
    only for inspection/debugging purposes. Feel free to use is if you need it.
    '''
    nodes = list()
    def build(v):
        if v not in nodes:
            nodes.insert(0, v)
            for child in (v._prev):
                build(child)
    build(node)
    return nodes

def print_graph(root):
    '''
    Produces a print out of the computational graph.
    '''
    nodes = trace_nodes(root)
    for n in nodes:
        print(n)

In [ ]:
# Here we give a graph visualization algorithm for your computational graph.
# You need to install the graphviz package: https://graphviz.org/
# After the installation, you are good to go. Feel free to play around
# with the visualization! For example, you can compare your own computational graph drawing.
from graphviz import Digraph

def trace(root):
  # builds a set of all nodes and edges in a graph
  nodes, edges = set(), set()
  def build(v):
    if v not in nodes:
      nodes.add(v)
      for child in v._prev:
        edges.add((child, v))
        build(child)
  build(root)
  return nodes, edges

def draw_graph(root):
  dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right
  
  nodes, edges = trace(root)
  for n in nodes:
    uid = str(id(n))
    # for any scalar in the graph, create a rectangular ('record') node for it
    dot.node(name = uid, label = "{ %s | scalar %s | grad %s }" % (n.name, n.scalar, n.grad), shape='record')
    if n._op:
      # if this scalar is a result of some operation, create an op node for it
      dot.node(name = uid + n._op, label = n._op)
      # and connect this node to it
      dot.edge(uid + n._op, uid)

  for n1, n2 in edges:
    # connect n1 to the op node of n2
    dot.edge(str(id(n1)), str(id(n2)) + n2._op)

  return dot

### 3.2) Test your implementation

Here, you need to implement the equations from the second exercise. You basically need to write down the intermediate values from ***2.a)***. The important part is that you need to make a gradient computation for these equations. As input scalars, you can choose whatever you want. However, first you need to make the computation per hand such that you can debug your code in case your implementation gives a different result. You can look at the gradient by printing either the node itself or __node.grad__

Let's make a simple example. Here is a tiny equation:
$$
\mathcal{Z} = a*b + c
$$

In [ ]:
# Implement the equation Z and compute the gradients here.
a = Node(2.0,  name='a')
b = Node(-3.0, name='b')
c = Node(10.0, name='c')

# Due to overloading the operators we can use the arithmetic
# operators like +, * directly in expressions. Note that the
# expression 
#
#     a * b + c
# 
# is equivalent to 
#
#     a * b + c = a.__mul__(b).__add__(c)
#
# Moreover, it automatically creates intermediate nodes for
# a * b and _ + c.
#
# In order to inspect every single piece of the graph, we 
# manually create intermediate nodes:

Z_t1 = a * b; Z_t1.name='Z_t1'
Z_t2 = Z_t1 + c; Z_t2.name='Z_t2'

# We run the backward function to compute all the gradients starting from the end node.
# The gradient of each node n is stored in n.grad
Z_t2.backward()

# Print the nodes to see their name, scalar and *most importantly* their gradients.
print(c)
print(a)
print(b)
print(Z_t1)
print(Z_t2)
print()

# We could have written this more compact (as explained above):
a.grad=0; b.grad=0; c.grad=0;
Z = a * b + c; Z.name='Z'
Z.backward()
print_graph(Z)
#
# Notes:
# - Due to the use of the data structure set, the nodes stored in
#   every _prev variable are not arranged in the order they have been
#   inserted -> this is why the printing order is c, a, b (don't worry
#   about it).
# - A repeated call of backward() will cause an accumulation of gradients
#   within the respective nodes. 
# 
#

In [ ]:
# With the following one-liner, you can visualize the comp. graph
draw_graph(Z)

After understanding the previous example, you can start the exercise! We write down the equations here to make your life easier.
$$
\mathcal{A} = \sqrt{a + b + c^2} + \log(a + b + c^2) + \frac{a + b + c^2}{bc^2}
$$

In [ ]:
# Implement the equation A and compute the gradients here.


$$
\mathcal{B} = \sum_{i=1}^{3}(w_0 + w_1x_i - y_i)^2
$$

In [ ]:
# Implement the equation B and compute the gradients here.
